# How to use PyARPES (private version)

## Introduction

There is a python library called [pyarpes](https://arpes.readthedocs.io/) for analyzing ARPES data.
It seems to be quite incomplete and has some buggy functions. However, it is still usable (or so I think).
I was wondering what is the difference between this and Igor's analysis?  I will end up using Igor, but it might be useful when handling a large amount of data.
Basically, it is designed to handle files that extend xarray, and xarray itself seems to be stable, so its reliability is relatively high.

Unfortunatelyy, this package is no longer maintained very well, and in many ways it is not a modern Pythonic description. I decided to manage it myself. Let's aim to make it a robust package.

[pyarpes](https://arpes.readthedocs.io/)といpython ライブラリがある。
かなり不完全なところがあって、バギーな関数もある様子。ただ、一応なりとも使える（かな）。
もっともIgorで行う解析と何が違うのか？という疑問はあって、結局Igorでやることになるとは思うのだけど、大量データを扱うことを考えたときに便利かもしれない。
というか、便利なのが分かったらIgor procedureを書かないといけないので、そのプロトタイプと考えると良いのかもしれない。
基本的に xarray を拡張したファイルを取り扱う設計で、 xarray そのものは安定しているようなので、信頼性は比較的高い。

もうかなりメンテされていないし、現代的なPythonの記述とはなっていない点も多い。自分で管理することにした。ロバストなパッケージにすることを目指そう。

## autoreload related

Eventually, following three lines are not essential.
最終的にはここは無くてもいい。

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## import 

Let's do the following:

* Well, we need numpy in any case.
* It's better to set matplotlib explicitly to easily tune the graph.
* For coloring the ARPES data by logarithmic scale, we introduce the LogNorm.
* arpes.config is (probably) required absolutely
* ~~pes.SPD_main.py is written by me, that why the the directory is not arpes~~ Now, SPD_main.py is moved to my pyarpes repository. 
* qt_tool is used to display the data by using PyQt.
* ktool is the same as above but for momentum analysis?
* convert_to_kspace is used to convert from angle to momentum. 
* hvplot.xarray may not be needed.  If you want to use Bokeh.

Note that you need enough libraries installed to load the above libraries and not get an error.

Currently, PyQt5 is required. (In the near future, I'll rewrite this.)

* (In Mac, installation by pip will fail. You need install by brew and after that do the following
* After ```brew install pyqt5``` 

```bash
ln -s /opt/homebrew/Cellar/pyqt@5/${pyqt_version}/lib/python${python_version}/site-packages/*  $(dirname $(which python))/../lib/python${python_version}/site-packages/
```
The above operation may be required.

In [ ]:
import arpes.config
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
from arpes.io import load_data
from arpes.plotting.qt_ktool import ktool
from arpes.plotting.qt_tool import qt_tool
from arpes.utilities.conversion import convert_to_kspace
from matplotlib.colors import LogNorm

## データのロード

自分で測定したデータを読み込むために、
* from arpes.io  import load_data 

が必須。

自分のデータの内容に不安を覚えたら、
* from arpes.io import example_data

を使い、

```python:
ex = example_data.cut
```

として、　ex の中身を見て比べてみると何かヒントがあるかもしれない。


In [ ]:
from arpes.io import example_data

load_data は以上のような関数で、実際に使うだけなら、そんなにむずかしくない。単純に

In [ ]:
f = load_data(
    "/Users/arafune/src/arpes/arpes/example_data/example_itx_data.itx",
    location="SPD_phoibos",
    trace=True,
)
# f2 = load_data("/Users/arafune/src/analysis_tool/test/pes/GrIr.sp2", location="SPD_phoibos")

でよい。ただし、

* x, y, z 軸の値や 角度βを記録に含めたい。

など、itx ファイルや、 sp2ファイルに記載されない情報も 付加させたいときは。

```python
f=load_data("/Users/arafune/src/analysis_tool/test/pes/PES_16_Spectrum_3.itx",location="SPD_phoibos",
             x=1,
             sample_name="Cu(111)")
```

などと、することができる。

''x'' や ''sample_name''　などの変数名は予め定義されているのでそれを使うように推奨される。（そうでないと、ただのコメントにしかならない）
どのような変数名が公式なものかは、どこにもリストらしきものはないので、実際に読み込んだ後で、チェックする必要があるかもしれない。

* sample_name
* sample_workfunction 

がちょっと知らないと使えない変数名。 ("sample" ではないし、 "workfunction"ではない。（アナライザーの仕事関数として予約されている） )
このマニュアルでは（少なくともこのセクションでは）付加情報は加えていない。

``xarray_extensions.py`` を読むしかない。

In [ ]:
f.S.polarization

f.attrs
```python
{'spectrum_type': 'cut',
 'Created Date (UTC)': '2022-Feb-28 05:03:33.001919',
 'Igor Text File Exporter Version': 0.3,
 'Created by': 'SpecsLab Prodigy, Version 4.86.2-r103043',
 'Scan Mode': 'Fixed Analyzer Transmission',
 'User Comment': '',
 'Analysis Mode': 'UPS',
 'Lens Voltage': '40V',
 'Analyzer Slits': '1:0.5x20\\B:open',
 'Number of Scans': 30,
 'Number of Samples': 501,
 'Scan Step': 0.002,
 'DwellTime': 0.096,
 'Kinetic Energy': 9,
 'Binding Energy': -4.008,
 'Bias Voltage': -1.5,
 'angle_unit': 'rad (theta_y)',
 'enegy_unit': 'eV (Kinetic Energy)',
 'count_unit': 'cps (Intensity)',
 'trace': Trace(silent=False, start_time=1688363879143870000),
 'hv': 4.992,
 'workfunction': 4.401,
 'lens_mode': 'WideAngleMode',
 'pass_energy': 5,
 'mcp_voltage': 1500,
 'id': 2,
 'x': nan,
 'y': nan,
 'z': nan,
 'theta': 0,
 'beta': 0,
 'chi': 0,
 'alpha': 1.5707963267948966,
 'energy_notation': 'Kinetic',
 'analyzer': 'Specs PHOIBOS 100',
 'analyzer_name': 'Specs PHOIBOS 100',
 'parallel_deflectors': False,
 'perpendicular_deflectors': False,
 'analyzer_radius': 100,
 'analyzer_type': 'hemispherical',
 'psi': 0,
 'chi_offset': 0}
 ```

f.S.spectrometer_settings
```python
{'lens_mode': 'WideAngleMode', 'pass_energy': 5}
```

f.S.experimental_conditions

```python
{'hv': 4.992, 'polarization': None, 'temp': None}
```

f.S.scan_info
```python
{'time': None,
 'date': None,
 'type': None,
 'spectrum_type': 'cut',
 'experimenter': None,
 'sample': None}
```

f.S.sample_angles
```python
(0,
 0,
 0,
 <xarray.DataArray 'phi' (phi: 600)>
 array([-0.217803, -0.217076, -0.216349, ...,  0.216349,  0.217076,  0.217803])
 Coordinates:
   * phi      (phi) float64 -0.2178 -0.2171 -0.2163 ... 0.2163 0.2171 0.2178
     x        float64 nan
     y        float64 nan
     z        float64 nan
     theta    int64 0
     beta     int64 0
     chi      int64 0
     hv       float64 4.992
     alpha    float64 1.571
     psi      int64 0,
 0,
 1.5707963267948966)
```


f.S.sample_pos
```python
(nan, nan, nan)
```
x, y, z

f.S.experiment_info
```python
{'temperature': None,
 'temperature_cryotip': None,
 'pressure': None,
 'polarization': (None, None),
 'photon_flux': None,
 'photocurrent': None,
 'probe': None,
 'probe_detail': None,
 'analyzer': 'Specs PHOIBOS 100',
 'analyzer_detail': {'name': 'Specs PHOIBOS 100',
  'parallel_deflectors': False,
  'perpendicular_deflectors': False,
  'type': 'hemispherical',
  'radius': 100}}
```


f.S.laser_info
```python
{'probe_wavelength': None,
 'probe_energy': 4.992,
 'probe_fluence': None,
 'probe_pulse_energy': None,
 'probe_spot_size': (None, None),
 'probe_profile': None,
 'probe_linewidth': None,
 'probe_temporal_width': None,
 'probe_polarization': (None, None),
 'pump_wavelength': None,
 'pump_energy': None,
 'pump_fluence': None,
 'pump_pulse_energy': None,
 'pump_spot_size': (None, None),
 'pump_profile': None,
 'pump_linewidth': None,
 'pump_temporal_width': None,
 'pump_polarization': (None, None),
 'repetition_rate': None}
```

f.S.probe_info
```python
{'probe_wavelength': None,
 'probe_energy': 4.992,
 'probe_fluence': None,
 'probe_pulse_energy': None,
 'probe_spot_size': (None, None),
 'probe_profile': None,
 'probe_linewidth': None,
 'probe_temporal_width': None,
 'probe_polarization': (None, None)}
```

f.S.pump_info
```python
{'pump_wavelength': None,
 'pump_energy': None,
 'pump_fluence': None,
 'pump_pulse_energy': None,
 'pump_spot_size': (None, None),
 'pump_profile': None,
 'pump_linewidth': None,
 'pump_temporal_width': None,
 'pump_polarization': (None, None)}
```

f.S.analyzer_info
```python
{'lens_mode': 'WideAngleMode',
 'lens_mode_name': None,
 'acquisition_mode': None,
 'pass_energy': 5,
 'slit_shape': None,
 'slit_width': None,
 'slit_number': None,
 'lens_table': None,
 'analyzer_type': 'hemispherical',
 'mcp_voltage': 1500,
 'work_function': 4.401}
```

## Plot

機能としてはシンプル。この辺は、 pyarpes の機能ではなく、 xarray の機能と考えるのが無難。
上述のごとくxarrayは比較的良く整備されているように見える。
ま、ただのデータ構造なので、メンテがそれほど難しくないのかもしれない。

### matplotlib base: S.plot()

In [ ]:
f.S.plot()

以下ようにすると、matplotlib.pyplot を直接使う事ができるように見える。

In [ ]:
fig = plt.figure(figsize=(4, 3))
ax = fig.add_subplot(111)
f.S.spectrum.T.plot(ax=ax)

しかし、xarray の plot method は matplotlib の thin wrapperなだけなので、出力フォーマットを変えるのであれば、以下のようにする方がよりよい。

In [ ]:
f.S.spectrum.T.plot(aspect=0.7, size=6, cmap=plt.cm.plasma)
plt.ylabel("Energy relative to E_F ( eV )")
plt.xlabel("Emission Angle ( rad )")
plt.title("Xe/Au(111) monochromatic 2PPE")
plt.show()

In [ ]:
# Logarithmic scale
f.S.spectrum.T.plot(aspect=0.7, size=4, cmap=plt.cm.jet, norm=LogNorm(vmin=1, vmax=100))
plt.ylabel("Energy relative to E_F ( eV )")
plt.xlabel("Emission Angle ( rad )")
plt.title("Xe/Au(111) monochromatic 2PPE")
plt.show()

### Matplotlib base: stack-trace

In [ ]:
from arpes.plotting import stack_plot

In [ ]:
int(np.log10(np.abs(-0.196)))+1
np.round(0.196,1)

In [ ]:
stack_plot.flat_stack_plot(f.S.spectrum, loc="upper left", color="Blues")

I have largely revised stack_dispersion_plot function.

The examples are shown below:

In [ ]:
fig1, ax1 = stack_plot.stack_dispersion_plot(
    f.S.spectrum,
    max_stacks=20,
    scale_factor=.3,
    title="2PPE Xe/Au(111)",
    linewidth=0.3,
    color="plasma",
    shift=0.00,
    mode="hide_line",
    label="label test",
    figsize=(7, 5),
)

In [ ]:
fig1, ax1 = stack_plot.stack_dispersion_plot(
    f.S.spectrum,
    max_stacks=10,
    scale_factor=.1,
    title="2PPE Xe/Au(111)",
    linewidth=0.5,
    color="plasma",
    shift=0.00,
    label="label test",
    mode="fill_between",
    figsize=(7, 5),
    offset_correction="zero",
)

In [ ]:
fig1, ax1 = stack_plot.stack_dispersion_plot(
    f.S.spectrum,
    max_stacks=130,
    title="2PPE Xe/Au(111)",
    linewidth=0.5,
    color="viridis",
    shift=0.00,
    label="label test",
    mode="hide_line",
    figsize=(7, 4),
)

In [ ]:
fig1, ax1 = stack_plot.stack_dispersion_plot(
    f.S.spectrum,
    max_stacks=130,
    title="2PPE Xe/Au(111)",
    linewidth=0.5,
    color="viridis",
    shift=0.00,
    label="label test",
    mode="hide_line",
    figsize=(7, 4),
)

#### Bokeh base 

Bokeh を使って、インタラクティブな可視化をすることもできる。直接Bokeh を使っても良いはずだろうとは思うけど、ここではhvplot を使ってみる。
~~ただし、hvplot の出力はvscode で見ることができない様なので注意。~~ 

In [ ]:
# f.hvplot.image(colorbar=True)

In [ ]:
# f.S.spectrum.T.hvplot(invert=True, frame_height=400, frame_width=200)

### QT base
(2023/06/09　動くようにできた)

In [ ]:
# qt_tool(f)

In [ ]:
# ktool(f)

In [ ]:
# f.S.show_band_tool() 今のところこれが動かない

## k-space 変換

`convert_to_kspace` というそのものズバリの関数がある。以下の例では、 `kp` を指定してあるが、もし指定しないと適切な範囲を自動的に決めてくれる。~~ただ、エネルギー軸をどうするかはこれから検証する必要がある。~~ いまでは、データがKinetic Energy (Final State Energy)としてエネルギー軸をもっているのか、Binding Energyとしてエネルギー軸を持っているのかを記録しておくようにして、それに合わせて自動的にかつ適切に θ-k するようにしてある。

In [ ]:
convert_to_kspace(f.spectrum, kp=np.linspace(-0.25, 0.25, 100)).plot()
plt.ylabel("Energy relative to E_F ( eV )")
plt.xlabel("$k_{//}$")

## まだよく分かってないこと。

* Fermi level correction から アナライザーの misalignment に由来するズレを解消できるか？
* attrs.offset や beta angle の設定は自動的に考慮されてしまうのか？

いずれにしても、これは個々のケースバイケースの話なので、今回はふれない（ MoTe2 ARPES をこのパッケージで解析すると自動的に上の二つの問題には直面することになる・・・）